In [ ]:
import sys
import pickle
import pandas as pd
import sklearn.metrics as skm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBRegressor
from sklearn.metrics import classification_report
import numpy as np
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import ConfusionMatrix


In [ ]:
%matplotlib auto

# Load Training Model and Test Data

In [ ]:
labels = ['Influenza', 'Covid-19', 'Other']
obj = pd.read_pickle(f'models/Model.pkl')
print(obj)
data = pd.read_csv(f"data/Data_Test.csv")
model = pickle.load(open(f"models/Model.pkl", 'rb'))"
print(skm.classification_report(data['Class'], model.predict(data.drop('Class', axis=1)), target_names=labels))

# Show confusion matrix

In [ ]:
y_test = data['Class']
X_test = data.loc[:, data.columns != 'Class']
y_score = model.predict_proba(X_test)
cm = ConfusionMatrix(model)
cm.score(X_test, y_test)
cm.ax.tick_params(labelsize=22)
cm.ax.title.set_fontsize(30)
for xtick,ytick in zip(cm.ax.xaxis.get_major_ticks(),cm.ax.yaxis.get_major_ticks()):
    xtick.label.set_fontsize(30) 
    ytick.label.set_fontsize(30)
for label in cm.ax.texts:
    label.set_size(24)
cm.show()


# Show ROC curve and AUC

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = skm.roc_curve((y_test == i).astype(int), y_score[:, i], )
    roc_auc[i] = skm.auc(fpr[i], tpr[i])
plt.figure(figsize=(8, 6))
plt.rcParams.update({'font.size': 22})
for i in range(3):
    plt.plot(fpr[i], tpr[i], label=f'Class {labels[i]} (AUC = %0.3f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('ROC curve',fontsize = 22)
    plt.xlabel('False positive rate',fontsize = 20)
    plt.ylabel('True positive rate',fontsize = 20)
    plt.legend(loc='best')
plt.show()